# Disney+ Movies and TV Shows

About this Dataset: Disney+ is another one of the most popular media and video streaming platforms. They have close to 1300 movies or tv shows available on their platform, as of mid-2021, they have over 116M Subscribers globally. This tabular dataset consists of listings of all the movies and tv shows available on Amazon Prime, along with details such as - cast, directors, ratings, release year, duration, etc.

In [1]:
import findspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as f
import datetime as dt

spark = SparkSession.builder.getOrCreate()

disney = spark.read.options(delimiter=',', quotes='""').csv(
    "disney_plus_titles.csv", header=True)

disney = disney.dropna()
disney = disney.drop("description", "cast", "release_year", "duration")

# disney = disney.withColumn('date_added', f.regexp_replace(
#     "date_added", ",", "")).drop(disney.date_added)
# disney = disney.withColumn('date_added', f.split(
#     "date_added", " ")).drop(disney.date_added)

# date = f.udf(lambda x : '{}-{}-{}'.format(
#     x[2], dt.datetime.strptime(x[0],"%B").month ,x[1]
# ))

# disney = disney.withColumn('date_added', date(disney.date_added))

test = f.udf(lambda x: "{}".format(x))

disney = disney.withColumn('date_added', test(disney.date_added))

disney = disney.withColumn('title', test(disney.title))

disney = disney.withColumn('director', f.split('director', ",")[0])

list_country = []

for i in disney[['country']].collect():
    list_country.append(i['country'])

list_country = set(",".join(list_country).split(","))
list_country = [*map(lambda x: x.strip(" "), list_country)]
list_country = list(set(list_country))

df_country = disney

df_country = df_country.withColumn(
    'country', f.regexp_replace('country', ', ', ',')).drop(df_country.country)

df_country = df_country.withColumn(
    'country', f.split('country', ",")).drop(df_country.country)

schemas = StructType(
    [StructField(i, StringType(), False) for i in disney.columns]
)
df_disney_by_country = spark.createDataFrame(data=[], schema=schemas)

for i in list_country:
    if i == 'Phylicia Rashad':
        result = df_country.filter(f.array_contains(
            f.col("country"), ' Phylicia Rashad'))

        result = result.withColumn('country',
                                   f.when(f.array_contains(
                                       f.col("country"), ' Phylicia Rashad'), i)).drop(result.country)
        df_disney_by_country = df_disney_by_country.union(result)
    else:
        result = df_country.filter(f.array_contains(
            f.col("country"), i))

        result = result.withColumn('country',
                                   f.when(f.array_contains(
                                       f.col("country"), i), i)).drop(result.country)
        df_disney_by_country = df_disney_by_country.union(result)
df_disney_by_country = df_disney_by_country.drop(df_disney_by_country.listed_in)

df_disney_by_country.coalesce(1).write.option("header", True).csv("df_disney_by_country")
df_disney_by_country.show()


list_category = []

for i in disney[['listed_in']].collect():
    list_category.append(i['listed_in'])

list_category = ",".join(list_category).split(",")
list_category = [*map(lambda x: x.strip(" "), list_category)]
list_category = list(set(list_category))

df_category = disney

df_category = df_category.withColumn(
    'listed_in', f.regexp_replace('listed_in', ', ', ',')).drop(df_category.listed_in)

df_category = df_category.withColumn(
    "listed_in", f.split("listed_in", ",")).drop(df_category.listed_in)

df_disney_by_category = spark.createDataFrame(data=[], schema=schemas)

for i in list_category:
    result = df_category.filter(f.array_contains(f.col('listed_in'), i))
    result = result.withColumn('listed_in', f.when(
        f.array_contains(f.col('listed_in'), i), i)).drop(result.listed_in)
    df_disney_by_category = df_disney_by_category.union(result)

df_disney_by_category = df_disney_by_category.drop(
    df_disney_by_category.country)

df_disney_by_category.coalesce(1).write.option("header", True).csv("df_disney_by_category")
df_disney_by_category.show()


22/10/29 23:44:01 WARN Utils: Your hostname, Yosies-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.202.2 instead (on interface en0)
22/10/29 23:44:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/29 23:44:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/29 23:44:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


22/10/29 23:44:30 WARN DAGScheduler: Broadcasting large task binary with size 1191.4 KiB


22/10/29 23:44:43 WARN DAGScheduler: Broadcasting large task binary with size 1001.2 KiB
22/10/29 23:44:44 WARN DAGScheduler: Broadcasting large task binary with size 1001.2 KiB
22/10/29 23:44:44 WARN DAGScheduler: Broadcasting large task binary with size 1001.2 KiB


+-------+-----+--------------------+-------------------+--------+------------------+------+
|show_id| type|               title|           director| country|        date_added|rating|
+-------+-----+--------------------+-------------------+--------+------------------+------+
|    s58|Movie|          Thumbelina|          Don Bluth|   Spain|  October 22, 2021|     G|
|   s318|Movie|           Ferdinand|    Carlos Saldanha|   Spain|   January 8, 2021|    PG|
|   s399|Movie| Disneynature Oceans|     Jacques Perrin|   Spain|September 25, 2020|     G|
|  s1443|Movie|        Tomorrowland|          Brad Bird|   Spain| September 3, 2021|    PG|
|   s131|Movie|      Fantastic Four|          Tim Story| Germany|   August 13, 2021| PG-13|
|   s228|Movie|Fantastic Four: R...|          Tim Story| Germany|       May 7, 2021|    PG|
|   s314|Movie|        Isle of Dogs|       Wes Anderson| Germany|  January 15, 2021| PG-13|
|   s629|Movie|Around the World ...|       Frank Coraci| Germany|  February 1, 2

+-------+-----+--------------------+--------------------+------------------+------+---------+
|show_id| type|               title|            director|        date_added|rating|listed_in|
+-------+-----+--------------------+--------------------+------------------+------+---------+
|    s56|Movie|      McFarland, USA|           Niki Caro|  October 29, 2021|    PG|   Sports|
|    s57|Movie|  Rookie of the Year|        Daniel Stern|  October 22, 2021|    PG|   Sports|
|   s187|Movie|       The Sandlot 2|  David Mickey Evans|      July 2, 2021|    PG|   Sports|
|   s344|Movie|              Safety|     Reginald Hudlin| December 11, 2020|    PG|   Sports|
|   s411|Movie|       Morning Light|         Mark Monroe|September 18, 2020|    PG|   Sports|
|   s485|Movie|       The Big Green|         Holly Sloan|      July 3, 2020|    PG|   Sports|
|   s486|Movie|    The Mighty Ducks|       Stephen Herek|      July 3, 2020|    PG|   Sports|
|   s608|Movie|The Sandlot: Head...|        William Dear|   

In [3]:
set(df_disney_by_category[['rating']].collect())


{Row(rating='G'),
 Row(rating='PG'),
 Row(rating='PG-13'),
 Row(rating='TV-14'),
 Row(rating='TV-G'),
 Row(rating='TV-PG'),
 Row(rating='TV-Y'),
 Row(rating='TV-Y7'),
 Row(rating='TV-Y7-FV')}

In [4]:
disney.count()

818